In [1]:
import numpy as np

# 创建测试数据集来测试函数
dataset = ['my dog has flea problems help please', 'maybe not take him to dog park stupid',
          'my dalmation is so cute I love him', 'stop posting stupid worthless garbage',
          'mr licks ate my steak how to stop him', 'quit buying worthless dog food stupid']
dataset = [sentence.split(' ') for sentence in dataset]
labels = [0,1,0,1,0,1]

testset = ['love my dalmation', 'stupid garbage']
testset = [s.split(' ') for s in testset]

# 朴素贝叶斯

朴素贝叶斯是一种使用概率论进行分类的方法。《机器学习实战》书中只讲了使用朴素贝叶斯进行文本分类，再加上初学嘛，先搞定经典运用场景。文本代表一串字符串，由很多单个的词组成。

## 1. 基本概念：贝叶斯和朴素

### 1.1 贝叶斯

朴素贝叶斯算法的核心思想就是**贝叶斯决策理论「选择具有最高概率的决策」**。举例子说明，假设是个二分类问题。$P1(W),P2(W)$ 分别代表文本 W 是类别 1 和类别 2 的概率。如果 $P1(W) > P2(W) $, 那么 $W$ 的类别是 1 ；反之，是 2 。

而 $P1(W),P2(W)$ 是一种简化的表达式，实际上文本 W 是类别 1 和类别 2 的概率的准确表达式为 $P(c_1|W),P(c_0|W)$。这是条件概率。通常情况下，容易求得 $P(W|c_1),P(W|c_2)$。

**贝叶斯准则**：公式如下，它告诉我们如何交换条件概率中的条件和结果。*（PS：其实可以通过条件概率公式推导得到）* $$P(c_1|W) = \frac{P(W|c_1) P(c_1)}{P(W)}$$

### 1.2文本的表示

文本 $W$ 是由多个单个的词 $w_i$ 组成的，即 $W=(w_0,w_1,...,w_n)$。一般文本 W 在计算机中用词向量表示。词向量又涉及到词汇表。

**词汇表**：将某个场景中的出现所有词组成在一起形成的。

举个例子，某个示例的词汇表为 `['hello', 'my', 'has', 'help', 'dog']`, `W = 'hello my dog dog'`,那么用词向量表示 `W=[1,1,0,0,1]`

上面的词向量表示称为**词集模型，即只统计该词是否在文本中出现**。算法见 `words2vec_1()`

还有一种词向量表示为**词袋模型，统计该词在文本中出现的次数**。此时词向量表示`W=[1,1,0,0,2]`。算法见`word2vec_2()`

In [2]:
def create_vocab_list(dataset):
    '''
    创建词汇表
    将 dataset 中出现的所有单词提取出来作为词汇表
    :param dataset : 已切分成单词的文档,每一行是一个句子
    :return : 语料库
    '''
    vocab_set = set()
    for sentence in dataset:
        vocab_set = vocab_set | set(sentence)  # 取并集
    return list(vocab_set)

def words2vec_1(input_vector, vocab_list):
    '''
    将文档 input_vector 转换成词向量表示
    采用词集模型
    
    举个例子：
    vocab_list = ['a','b','c'], input_vector = ['b', 'a']，那么该输入向量的特征向量为[1,1,0]
    
    :param vocab_list : 语料库。
    :param input_vector : 输入样本向量。
    :return : input_vector 的特征向量 / 词向量
    '''
    feature_vector = [0]*len(vocab_list)
    for word in set(input_vector):
        if word in vocab_list:
            feature_vector[vocab_list.index(word)] = 1
        else:
            print('the word "{}" is not in my vocabulary!'.format(word))
    return feature_vector

def words2vec_2(input_vector, vocab_list):
    '''
    将文档 input_vector 转换成词向量表示
    采用词袋模型
    
    举个例子：
    vocab_list = ['a','b','c'], input_vector = ['b', 'a', 'b']，那么该输入向量的特征向量为[1,2,0]
    
    :param vocab_list : 语料库。
    :param input_vector : 输入样本向量。
    :return : input_vector 的特征向量 / 词向量
    '''
    feature_vector = [0]*len(vocab_list)
    # for word in set(input_vector)  # 词集模型
    for word in input_vector:  # 词袋模型
        if word in vocab_list:
            # feature_vector[vocab_list.index(word)] = 1 # 词集模型，以每个词的出现与否作为一个特征
            feature_vector[vocab_list.index(word)] += 1  # 词袋模型，以每个词在文档/句子的出现次数作为一个特征
        else:
            print('the word "{}" is not in my vocabulary!'.format(word))
    return feature_vector

#==========================测试代码================================
vocab_list = create_vocab_list(dataset)
print('词汇表：', vocab_list)

temp_sentence = ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid', 'not', 'to']
print('the sentence is {}'.format(temp_sentence))
print('词集模型：the vector is {}'.format(words2vec_1(temp_sentence, vocab_list)))
print('词袋模型：the vector is {}'.format(words2vec_2(temp_sentence, vocab_list)))


词汇表： ['licks', 'buying', 'not', 'stupid', 'has', 'I', 'garbage', 'posting', 'worthless', 'problems', 'dog', 'him', 'to', 'flea', 'is', 'how', 'take', 'cute', 'park', 'quit', 'stop', 'love', 'food', 'my', 'ate', 'help', 'so', 'please', 'maybe', 'steak', 'dalmation', 'mr']
the sentence is ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid', 'not', 'to']
词集模型：the vector is [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
词袋模型：the vector is [0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


### 1.3 朴素

**朴素**：是因为在整个过程中只做最原始、最简单的假设。

- 假设1 ：所有词都是互相独立的。首先，文本是由多个词组成的，则有 $P(W|c_i) = P(w_0,w_1,...,w_n|c_i)$。而$P(w_0,w_1,...,w_n|c_i)$ 很难求解。有了假设 1 后，$$P(W|c_i) = P(w_0,w_1,...,w_n|c_i) = P(w_0|c_i)P(w_1|c_i)\cdot \cdot \cdot P(w_n|c_i)$$ 意味着 $P(W|c_i)$ 等于每个词的条件概率 $P(w_i|c_i)$ 相乘，而 $P(w_i|c_i)$ 很容易得到。
- 假设2 ：文本的每个特征同等重要。（TODO：还没理解）

## 2. 训练朴素贝叶斯分类器


**目标**：对于新文本 $W$，预测它属于哪个类别 $c_i$。公式表示: $P(W|c_i)$ ，取其最大值的类别 $c_i$ 作为 $W$ 的预测类别。

**如何计算** $P(W|c_i)$ : 运用贝叶斯准则和假设 1 得 $$P(c_1|W) = \frac{P(W|c_1) P(c_1)}{P(W)} = \frac{P(w_0|c_i)P(w_1|c_i)\cdot \cdot \cdot P(w_n|c_i) P(c_1)}{P(w_0)P(w_1)\cdot \cdot \cdot P(w_n)}$$ 最后，转换为计算 $P(w_i|c_i),P(c_i)$ 。

**如何计算** $P(w_i|c_i),P(c_i)$ : 根据大数定理，当数据集足够大，概率可用频率表示。

**训练分类器**：就是通过训练集计算 $P(w_i|c_i),P(c_i)$ 的过程。算法见 `trainNB0(),trainNB1()`

**分类器的结果**：得到 $P(w_i|c_i),P(c_i)$。

下面是对于一个二分类问题的训练朴素贝叶斯分类器的过程。具体的需求：为文本进行分类，若文本待有侮辱性，则 $c=1$；反之，则 $c=0$。

In [3]:
def trainNB0(trainset):
    '''
    训练朴素贝叶斯分类器，对一文档/句子是否是侮辱文档/句子
    :param dataset : [np.array]训练集，包括了标签。维度(m,n+1)。m是样本数，n是语料库大小（因为是one-hot编码）
    :return : prob1_vector, prob0_vector, prob_abusive
    '''
    m = trainset.shape[0]
    n = trainset.shape[1] - 1
    prob_abusive = np.sum(trainset[:,-1]) / m  # 计算侮辱样本占总样本的比例, P(c_i = 1)
    prob1_num = np.zeros((1, n))  # 统计所有侮辱样本中语料库每个词出现的次数
    prob0_num = np.zeros((1, n))  # 统计所有非侮辱样本中语料库每个词出现的次数
    prob1_denom = 0.0  # 所有侮辱样本出现的单词总数
    prob0_denom = 0.0  # 所有非侮辱样本出现的单词总数

    for sample in trainset:  # 遍历每个样本
        if sample[-1] == 1:  # 第 i 个样本是侮辱样本
            prob1_num += sample[:-1]
            prob1_denom += np.sum(sample[:-1])
        else:
            prob0_num += sample[:-1]
            prob0_denom += np.sum(sample[:-1])

    prob0_vector = prob0_num / prob0_denom  # 代表每个单词在非侮辱性文档出现的频率。根据大数定理，频率可代表概率即 P(w_i | c_i = 0)
    prob1_vector = prob1_num / prob1_denom  # 代表每个单词在侮辱性样本出现的概率,可借此判断每个词的词性是否带有侮辱性.P(w_i | c_i = 1)
    return prob1_vector, prob0_vector, prob_abusive

#==============================测试代码====================================
trainset = np.zeros((len(dataset), len(vocab_list)+1))  # 训练集，维度为(m,n+1)
for i in range(len(dataset)):
    trainset[i,:-1] = words2vec_1(dataset[i], vocab_list)
    trainset[i, -1] = labels[i]

p1_v_0, p0_v_0, pA_0 = trainNB0(trainset)

print('发现文档是侮辱类的概率P(c=1) ', pA_0)
print('每个单词在侮辱性文档出现的概率P(w_i|c=1)', p1_v_0)
print('每个单词在非侮辱性文档出现的概率P(w_i|c=0)', p0_v_0)


发现文档是侮辱类的概率P(c=1)  0.5
每个单词在侮辱性文档出现的概率P(w_i|c=1) [[0.         0.05263158 0.05263158 0.15789474 0.         0.
  0.05263158 0.05263158 0.10526316 0.         0.10526316 0.05263158
  0.05263158 0.         0.         0.         0.05263158 0.
  0.05263158 0.05263158 0.05263158 0.         0.05263158 0.
  0.         0.         0.         0.         0.05263158 0.
  0.         0.        ]]
每个单词在非侮辱性文档出现的概率P(w_i|c=0) [[0.04166667 0.         0.         0.         0.04166667 0.04166667
  0.         0.         0.         0.04166667 0.04166667 0.08333333
  0.04166667 0.04166667 0.04166667 0.04166667 0.         0.04166667
  0.         0.         0.04166667 0.04166667 0.         0.125
  0.04166667 0.04166667 0.04166667 0.04166667 0.         0.04166667
  0.04166667 0.04166667]]


### 2.2 改良训练算法

上面训练算法的细节部分有两处需要修改：

1. 利用贝叶斯分类器对文本进行分类时，要计算多个概率的乘积以获得文本属于某个类别的概率，即计算 $P(w_0|1)P(w_1|1)\cdot \cdot \cdot P(w_N|1)$。如果其中一个概率值为 0 ，那么最后的乘积也为 0。为了降低这种影响，可以将所有词的出现数初始化为 1，并将分母初始化为 2 （对应下面第15-18行）

2. 容易出现下溢出。这是由于太多很小的数相乘造成的。当计算乘积 $P(w_0|1)P(w_1|1)\cdot \cdot \cdot P(w_N|1)$ 时，由于大部分因子都非常小，就会越乘越小，小到一定范围计算机四舍五入会得到 0 。对策是对乘积取自然对数（对应第28-29行），即 $$In(P(w_0|1)P(w_1|1)\cdot \cdot \cdot P(w_N|1)) = InP(w_0|1) + \cdot \cdot\cdot + InP(w_N|1) $$

In [4]:
def trainNB1(trainset):
    '''
    (改)训练朴素贝叶斯分类器，对一文档/句子是否是侮辱文档/句子

    trainNB0() 存在两个问题：
    1. P(w|1) = P(w_1|1)*..P(w_n|1) 如果其中有一个概率为0，就导致结果为0.
    2. 下溢出, 很多值很小的数相乘会越来越小,当小到一定程度时,python会四舍五入为0

    :param dataset : [np.array]训练集，包括了标签。维度(m,n+1)。m是样本数，n是语料库大小（因为是one-hot编码）
    :return : prob1_vector, prob0_vector, prob_abusive
    '''
    m = trainset.shape[0]
    n = trainset.shape[1] - 1
    prob_abusive = np.sum(trainset[:,-1]) / m  # 计算侮辱样本占总样本的比例, P(c_i = 1)
    prob1_num = np.ones((1, n))  # 统计所有侮辱样本中语料库每个词出现的次数 （修改处,解决问题1）
    prob0_num = np.ones((1, n))  # 统计所有非侮辱样本中语料库每个词出现的次数 （修改处,解决问题1）
    prob1_denom = 2.0  # 所有侮辱样本出现的单词总数 （修改处,解决问题1）
    prob0_denom = 2.0  # 所有非侮辱样本出现的单词总数 （修改处,解决问题1）

    for sample in trainset:  # 遍历每个样本
        if sample[-1] == 1:  # 第 i 个样本是侮辱样本
            prob1_num += sample[:-1]
            prob1_denom += np.sum(sample[:-1])
        else:
            prob0_num += sample[:-1]
            prob0_denom += np.sum(sample[:-1])

    prob0_vector = np.log(prob0_num / prob0_denom)  # 代表每个单词在非侮辱性文档出现的概率,即 P(w_i | c_i = 0) （修改处,解决问题2）
    prob1_vector = np.log(prob1_num / prob1_denom)  # 代表每个单词在侮辱性样本出现的概率,可借此判断每个词的词性是否带有侮辱性.P(w_i | c_i = 1) （修改处,解决问题2）    
    return prob1_vector, prob0_vector, prob_abusive

p1_v_1, p0_v_1, pA_1 = trainNB1(trainset)
print('发现文档是侮辱类的概率P(c=1) ', pA_1)
print('每个单词在侮辱性文档出现的概率P(w_i|c=1)', p1_v_1)
print('每个单词在非侮辱性文档出现的概率P(w_i|c=0)', p0_v_1)

发现文档是侮辱类的概率P(c=1)  0.5
每个单词在侮辱性文档出现的概率P(w_i|c=1) [[-3.04452244 -2.35137526 -2.35137526 -1.65822808 -3.04452244 -3.04452244
  -2.35137526 -2.35137526 -1.94591015 -3.04452244 -1.94591015 -2.35137526
  -2.35137526 -3.04452244 -3.04452244 -3.04452244 -2.35137526 -3.04452244
  -2.35137526 -2.35137526 -2.35137526 -3.04452244 -2.35137526 -3.04452244
  -3.04452244 -3.04452244 -3.04452244 -3.04452244 -2.35137526 -3.04452244
  -3.04452244 -3.04452244]]
每个单词在非侮辱性文档出现的概率P(w_i|c=0) [[-2.56494936 -3.25809654 -3.25809654 -3.25809654 -2.56494936 -2.56494936
  -3.25809654 -3.25809654 -3.25809654 -2.56494936 -2.56494936 -2.15948425
  -2.56494936 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -2.56494936
  -3.25809654 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -1.87180218
  -2.56494936 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -2.56494936
  -2.56494936 -2.56494936]]


## 3. 如何使用分类器进行分类

**如何使用分类器进行分类** ：对于新样本，利用 $P(w_i|c_i),P(c_i)$，就能计算出 $P(W|c_i)$，就能预测新样本的类别。

下面有两个分类函数。那是因为在改良的训练算法中我们取了对数。那么求解变成 $InP(W|c_i)$ $$InP(c_i|W) = \frac{In(P(W|c_i) P(c_i))}{In(P(W))} = \frac{InP(w_0|c_i) + \cdot \cdot\cdot + InP(w_N|c_i) + InP(c_i)}{InP(w_0) + \cdot \cdot\cdot + InP(w_N)}$$ 。

使用下面哪个分类函数取决于训练方法。`trainNB0()` 对应 `classify0()`，`trainNB1()` 对应 `classify1()`

PS：实际计算 $P(W|c_i)$ 中，由于分母不管 $c_i$ 都是一样的，所以一般仅计算分子部分然后比较即可。

In [5]:
def classify0(inX, prob0_vector, prob1_vector, prob_abusive):
    '''
    (仅适用于trainNB0得到的结果,因为p1,p0计算用的是np.sum)
    利用朴素贝叶斯训练得到的三个概率。计算 P(inX | c_i) 概率,并取最大值作为 inX 的类别
    :param inX : [np.array(1,n)] 测试样本的词向量
    :param prob1_vector : [np.array(1,n)] 代表每个单词在侮辱性样本出现的概率,可借此判断每个词的词性是否带有侮辱性.P(w_i | c_i = 1)
    :param prob0_vector : [np.array(1,n)] 代表每个单词在非侮辱性样本出现的概率,可借此判断每个词的词性是否带有非侮辱性.P(w_i | c_i = 0)
    :param prob_abusive : [float] 代表侮辱样本出现概率
    '''
    assert (prob1_vector.shape == prob0_vector.shape)
    assert (inX.shape == prob1_vector.shape)
    p1 = 1.0  # inX 类别为 1 的概率
    vector = inX * prob1_vector
    for i in range(vector.shape[1]):
        if inX[0,i] == 1:
            p1 *= vector[0,i]
    p1 *= prob_abusive
    p0 = 1.0  # inX 类别为 0 的概率
    vector = inX * prob0_vector
    for i in range(vector.shape[1]):
        if inX[0,i] == 1:
            p0 *= vector[0, i]
    p0 *= prob_abusive
    # print(p1,p0)
    predict_class = 1 if p1 > p0 else 0
    return predict_class


def classify1(inX, prob0_vector, prob1_vector, prob_abusive):
    '''
    (仅适用于trainNB1得到的结果,因为p1,p0计算用的是np.sum)
    利用朴素贝叶斯训练得到的三个概率。计算 P(inX | c_i) 概率,并取最大值作为 inX 的类别
    :param inX : 测试样本的词向量
    :param prob1_vector : 代表每个单词在侮辱性样本出现的概率,可借此判断每个词的词性是否带有侮辱性.P(w_i | c_i = 1)
    :param prob0_vector : 代表每个单词在非侮辱性样本出现的概率,可借此判断每个词的词性是否带有非侮辱性.P(w_i | c_i = 0)
    :param prob_abusive : 代表侮辱样本出现概率
    '''
    assert (prob1_vector.shape == prob0_vector.shape)
    assert (inX.shape == prob1_vector.shape)
    p1 = np.sum(inX * prob1_vector) + np.log(prob_abusive)  # inX 类别为 1 的概率。其中np.sum()是因为在trainNB1()取了对数
    p0 = np.sum(inX * prob0_vector) + np.log(1 - prob_abusive)  # inX 类别为 0 的概率
    # print(p1, p0)
    predict_class = 1 if p1 > p0 else 0
    return predict_class

#==========================测试代码================================
test_matrix = np.zeros((len(testset), len(vocab_list)))
for i in range(test_matrix.shape[0]):  # 将测试集每个样本转换为词向量
    test_matrix[i, :] = words2vec_1(testset[i], vocab_list)
for i in range(test_matrix.shape[0]):  # 循环每个样本
    print(testset[i], ' classified as ', classify0(test_matrix[i, :].reshape(p0_v_0.shape),
                                                   p0_v_0, p1_v_0, pA_0))
    print(testset[i], ' classified as ', classify1(test_matrix[i, :].reshape(p0_v_1.shape),
                                                   p0_v_1, p1_v_1, pA_1))

['love', 'my', 'dalmation']  classified as  0
['love', 'my', 'dalmation']  classified as  0
['stupid', 'garbage']  classified as  1
['stupid', 'garbage']  classified as  1


## 4. 示例：使用朴素贝叶斯过滤垃圾邮件

下面示例，是一个完整的朴素贝叶斯分类进行分类的过程：

1. 读入数据，并预处理数据（主要是切分文本为词）
2. 构建词汇表
3. 用词向量表示文本
4. 用训练集训练分类器
5. 对测试集测试分类器效果

In [6]:
# 数据预处理并准备数据
def textParse(bigString):
    '''
    将字符串按照一定规律切分成若干个词
    '''
    import re
    pattern = re.compile(r'\w*\b')  # 最简单的正则化
    list_of_tokens = pattern.findall(bigString)
    word_list = [tok.lower() for tok in list_of_tokens if len(tok) > 2]
    return  word_list# 长度小于 2 的词直接忽略

label_list = []  # 标签向量
dataset = []  # 数据集，二维列表。每行代表一个邮件
for i in range(1,26):  # 垃圾邮件和非垃圾邮件各26封，依次从文件中读取数据
    word_list = textParse(open('./dataset/email/spam/%d.txt' %i).read())  # 从一个垃圾邮件文件中读入数据并切分为若干个词
    dataset.append(word_list)
    label_list.append(1)
    word_list = textParse(open('./dataset/email/ham/%d.txt' %i).read())  # 从一个垃圾邮件文件中读入数据并切分为若干个词
    dataset.append(word_list)
    label_list.append(0)
print('文件读取完毕...')
vocab_list = create_vocab_list(dataset)  # 语料库
print('词汇表构建完毕...')


# 随机构建训练集和测试集
import random
train_indexs = [i for i in range(50)]  # 并不是一般定义的训练集，存放的是dataset索引号
test_indexs = []  # 并不是一般定义的测试集，存放的是dataset索引号
for i in range(10):
    rand_index = int(random.uniform(0, len(train_indexs)))
    test_indexs.append(rand_index)
    train_indexs[rand_index] = -1
for i in range(len(set(test_indexs))):
    train_indexs.remove(-1)
# print(train_indexs, test_indexs)
print('训练集和测试集构建完毕...')

# 先转换为词向量，后训练分类器
train_matrix = np.zeros((len(train_indexs), len(vocab_list)+1))
index = 0
for i in train_indexs:
    train_matrix[index,:-1] = words2vec_1(input_vector=dataset[i], vocab_list=vocab_list)
    train_matrix[index,-1] = label_list[i]
    index += 1
# print(train_matrix)
p1_v, p0_v, p_A = trainNB1(train_matrix)
print('训练完毕...')

# 进行分类
error_count = 0
for i in set(test_indexs):
    word_vector = words2vec_1(input_vector=dataset[i], vocab_list=vocab_list)
    predict_class = classify1(inX=np.array(word_vector).reshape(p0_v.shape), prob0_vector=p0_v, prob1_vector=p1_v, prob_abusive=p_A)
#     print('the real class is {} and the predicted class is {}'.format(label_list[i], predict_class))
    if predict_class != label_list[i]:
        print('the real class is {} and the predicted class is {}'.format(label_list[i], predict_class))
        print('the error is No.{} sample and the content is {}'.format(i, dataset[i]))
        error_count +=1

print('the error rate is ', error_count / float(len(test_indexs)))

文件读取完毕...
词汇表构建完毕...
训练集和测试集构建完毕...
训练完毕...
the real class is 0 and the predicted class is 1
the error is No.45 sample and the content is ['scifinance', 'now', 'automatically', 'generates', 'gpu', 'enabled', 'pricing', 'risk', 'model', 'source', 'code', 'that', 'runs', '300x', 'faster', 'than', 'serial', 'code', 'using', 'new', 'nvidia', 'fermi', 'class', 'tesla', 'series', 'gpu', 'scifinance', 'derivatives', 'pricing', 'and', 'risk', 'model', 'development', 'tool', 'that', 'automatically', 'generates', 'and', 'gpu', 'enabled', 'source', 'code', 'from', 'concise', 'high', 'level', 'model', 'specifications', 'parallel', 'computing', 'cuda', 'programming', 'expertise', 'required', 'scifinance', 'automatic', 'gpu', 'enabled', 'monte', 'carlo', 'pricing', 'model', 'source', 'code', 'generation', 'capabilities', 'have', 'been', 'significantly', 'extended', 'the', 'latest', 'release', 'this', 'includes']
the error rate is  0.1


PS : 误把正常邮件当作垃圾邮件比误把垃圾邮件当作正常邮件更不好.

## 5. 总结

优点：

1. 在数据较少的情况下仍然有效
2. 可以处理多类别问题
3. 贝叶斯分类法适用于任意的分类场景

缺点：

1. 分类器的性能跟词的切割有着很大的关系，即对输入数据的准备方式较为敏感

一些思考：
1. 用词向量表示，也就是忽略文本中词的顺序影响
2. （书中讲的）假设1：独立性，让算法变得很简单，但是实际上算法效果依旧很好。


